# 모듈 설치하기

```

!pip install pretty_midi librosa
!pip install moviepy


```

- ffmpeg 설치하기


In [1]:
!ffmpeg 

ffmpeg version 6.0-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libvpl --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-lib

# 동영상 파일을 1초단위로 자르기

In [1]:
from moviepy.editor import VideoFileClip

# 비디오 길이 리턴
def get_duration(filename):
    clip = VideoFileClip(filename)
    return clip.duration
    
time_list = [] # 1초 단위 시간 분할.

video_path = 'video1.mp4' # 비디오 경로 지정

video_time = int(get_duration(video_path))

m = 0
s = 0
for i in range(video_time):
    if s == 60:
        s = 0
        m += 1
    s+=1
    time_list.append(str(m)+':'+str(s))
    
print(time_list) # ffmpeg 시간 인자값에 대입 해주기 위해서

['0:1', '0:2', '0:3', '0:4', '0:5', '0:6', '0:7', '0:8', '0:9', '0:10', '0:11', '0:12', '0:13', '0:14', '0:15', '0:16', '0:17', '0:18', '0:19', '0:20', '0:21', '0:22', '0:23', '0:24', '0:25', '0:26', '0:27', '0:28', '0:29', '0:30', '0:31', '0:32', '0:33', '0:34', '0:35', '0:36', '0:37', '0:38', '0:39', '0:40', '0:41', '0:42', '0:43', '0:44', '0:45', '0:46', '0:47', '0:48', '0:49', '0:50', '0:51', '0:52', '0:53', '0:54', '0:55', '0:56', '0:57', '0:58', '0:59', '0:60', '1:1', '1:2', '1:3', '1:4', '1:5', '1:6', '1:7', '1:8', '1:9', '1:10', '1:11', '1:12', '1:13', '1:14', '1:15', '1:16', '1:17', '1:18', '1:19', '1:20', '1:21', '1:22', '1:23', '1:24', '1:25', '1:26', '1:27', '1:28', '1:29', '1:30', '1:31', '1:32', '1:33', '1:34', '1:35', '1:36', '1:37', '1:38', '1:39', '1:40', '1:41', '1:42', '1:43', '1:44', '1:45', '1:46', '1:47', '1:48', '1:49', '1:50', '1:51', '1:52', '1:53', '1:54', '1:55', '1:56', '1:57', '1:58', '1:59', '1:60', '2:1', '2:2', '2:3', '2:4', '2:5', '2:6', '2:7', '2:8', '

# !ffmpeg -i 원본파일 -ss 시작시간 -to 끝시간  출력파일

In [3]:
for i in range(len(time_list)):
    data_name = './example/output'+str(i)+'.mp4'
    !ffmpeg -i {video_path} -ss {time_list[i]} -to {time_list[i+1]} {data_name}

ffmpeg version 6.0-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libvpl --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame --enable-libtheora --enable-libvo-amrwbenc --enable-lib

In [16]:
import librosa
import pretty_midi
import numpy as np

# 오디오 to MIDI

def audio_to_midi(audio_file_path, midi_file_path, sr=22050, hop_length=512):
    y, sr = librosa.load(audio_file_path, sr=sr)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr, hop_length=hop_length)
    pitches = pitches[magnitudes > np.median(magnitudes)]
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    for pitch in np.unique(pitches):
        midi_pitch = int(librosa.hz_to_midi(pitch))
        if 0 <= midi_pitch <= 127:
            note = pretty_midi.Note(
                velocity=100, pitch=midi_pitch, start=0, end=1
            )
            instrument.notes.append(note)
    midi.instruments.append(instrument)
    midi.write(midi_file_path)

audio_file_path = 'output11.wav' 
midi_file_path = 'output11.mid' 

audio_to_midi(audio_file_path, midi_file_path)


In [17]:
import pretty_midi
import os

# midi 1초 단위 분할.

def split_midi(file_path, output_dir, split_length=1.0):
    midi_data = pretty_midi.PrettyMIDI(file_path)
    total_length = midi_data.get_end_time()
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for start in range(int(total_length)):
        split_midi = pretty_midi.PrettyMIDI()
        for instrument in midi_data.instruments:
            split_instrument = pretty_midi.Instrument(program=instrument.program, is_drum=instrument.is_drum)
            for note in instrument.notes:
                if start <= note.start < start + split_length:
                    new_note = pretty_midi.Note(
                        velocity=note.velocity,
                        pitch=note.pitch,
                        start=note.start - start,
                        end=min(note.end, start + split_length) - start
                    )
                    split_instrument.notes.append(new_note)
            if split_instrument.notes:
                split_midi.instruments.append(split_instrument)
        split_file_path = os.path.join(output_dir, f'split_{start}.mid')
        split_midi.write(split_file_path)


for i in range(video_time):
    file_path = './video1.mid' 
    output_dir = './example_midi'
    split_midi(file_path, output_dir)
